# Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models

This notebook shows an example for the threat model and attack described in "Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models
". This example deviates from the other "honest-but-curious" server models and investigates a malicious server that may send malicious server updates. The attack succeeds for a range of common transformer architectures and works merely by sending a single malicious query to the user model.

In this notebook, we attack the (small) GPT-2 model (120mil parameters).



Paper URL: https://arxiv.org/abs/2201.12675

### Abstract:
A central tenet of Federated learning (FL), which trains models without centralizing user data, is privacy. However, previous work has shown that the gradient updates used in FL can leak user information. While the most industrial uses of FL are for text applications (e.g. keystroke prediction), nearly all attacks on FL privacy have focused on simple image classifiers. We propose a novel attack that reveals private user text by deploying malicious parameter vectors, and which succeeds even with mini-batches, multiple users, and long sequences. Unlike previous attacks on FL, the attack exploits characteristics of both the Transformer architecture and the token embedding, separately extracting tokens and positional embeddings to retrieve high-fidelity text. This work suggests that FL on text, which has historically been resistant to privacy attacks, is far more vulnerable than previously thought.

### Startup

In [1]:
try:
    import breaching
except ModuleNotFoundError:
    # You only really need this safety net if you want to run these notebooks directly in the examples directory
    # Don't worry about this if you installed the package or moved the notebook to the main directory.
    import os; os.chdir("..")
    import breaching
    
import torch
%load_ext autoreload
%autoreload 2

# Redirects logs directly into the jupyter notebook
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will load the full configuration object. This includes the configuration for the use case and threat model as `cfg.case` and the hyperparameters and implementation of the attack as `cfg.attack`. All parameters can be modified below, or overriden with `overrides=` as if they were cmd-line arguments.

In [2]:
cfg = breaching.get_config(overrides=["attack=decepticon", "case=10_causal_lang_training", 
                                     "case/server=malicious-transformer"]) # optional: "case/data=stackoverflow"
          
device = torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=getattr(torch, cfg.case.impl.dtype))
setup

Investigating use case causal_lang_training with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations for the attack, or the case:

In [3]:
cfg.case.user.num_data_points = 8 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [512] # This is the sequence length

cfg.case.server.provide_public_buffers = True # Send server signal to disable dropout
cfg.case.server.has_external_data = True  # Not strictly necessary, but could also use random text (see Appendix)
cfg.case.data.tokenizer = "gpt2"
cfg.case.model = "gpt2" # Could also choose "gpt2S" which contains ReLU activations

## Attack hyperparameters:

# Server side:
cfg.case.server.param_modification.v_length = 32 # Length of the sentence component
cfg.case.server.param_modification.eps = 1e-8
cfg.case.server.param_modification.measurement_scale=1e6 # Circumvent GELU
cfg.case.server.param_modification.imprint_sentence_position = 0
cfg.case.server.param_modification.softmax_skew = 1e8
cfg.case.server.param_modification.sequence_token_weight = 1

# Attacker side:

# this option requires installation of `k-means-constrained` which can be tricky:
# If this doesn't work for you, falling back to "dynamic-threshold" is still a decent option.
cfg.attack.sentence_algorithm = "k-means" 
cfg.attack.token_strategy="embedding-norm" # no decoder bias in GPT
cfg.attack.embedding_token_weight=0.0 # Setting e.g. 0.25 here can improve performance slightly for long sequences

### Instantiate all parties

The following lines generate "server, "user" and "attacker" objects and print an overview of their configurations.

In [4]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Model architecture gpt2 loaded with 124,439,808 parameters and 12,582,924 buffers.
Overall this is a data ratio of   30381:1 for target shape [8, 512] given that num_queries=1.
User (of type UserSingleStep) with settings:
    Number of data points: 8

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: wikitext
    user: 1
    
        
Server (of type MaliciousTransformerServer) with settings:
    Threat model: Malicious (Parameters)
    Number of planned queries: 1
    Has external/public data: True

    Model:
        model specification: gpt2
        model state: default
        public buffers: True

    Sec

### Simulate an attacked FL protocol

This exchange is a simulation of a single query in a federated learning protocol. The server sends out a `server_payload` and the user computes an update based on their private local data. This user update is `shared_data` and contains, for example, the parameter gradient of the model in the simplest case. `true_user_data` is also returned by `.compute_local_updates`, but of course not forwarded to the server or attacker and only used for (our) analysis.

In [5]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)  

Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external data.
Feature mean is -27128.7578125, feature std is 728460.3125.
Computing user update on user 1 in model mode: eval.


In [6]:
user.print(true_user_data)

 The Tower Building of the Little Rock Arsenal, also known as U.S. Arsenal Building, is a building located in MacArthur Park in downtown Little Rock, Arkansas. Built in 1840, it was part of Little Rock's first military installation. Since its decommissioning, The Tower Building has housed two museums. It was home to the Arkansas Museum of Natural History and Antiquities from 1942 to 1997 and the MacArthur Museum of Arkansas Military History since 2001. It has also been the headquarters of the Little Rock Æsthetic Club since 1894. 
 The building receives its name from its distinct octagonal tower. Besides being the last remaining structure of the original Little Rock Arsenal and one of the oldest buildings in central Arkansas, it was also the birthplace of General Douglas MacArthur, who became the supreme commander of US forces in the South Pacific during World War II. It was also the starting place of the Camden Expedition. In 2011 it was named as one of the top 10 attractions in the s

### Reconstruct user data:

Now we launch the attack, reconstructing user data based on only the `server_payload` and the `shared_data`. 

For this attack, we also share secret information from the malicious server with the attack (`server.secrets`), which here is the location and structure of the imprint block.

In [7]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], server.secrets, 
                                                      dryrun=cfg.dryrun)

Proceeded to cut estimated token distribution at 1.50.
Recovered tokens tensor([   11,    11,    11,  ..., 50203, 50210, 50210]) through strategy embedding-norm.
Recovered 4388 embeddings with positional data from imprinted layer.
Reduced to 4096 hits.
Assigned [512, 512, 512, 512, 512, 512, 512, 512] breached embeddings to each sentence.


Next we'll evaluate metrics, comparing the `reconstructed_user_data` to the `true_user_data`.

In [8]:
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, order_batch=True, compute_full_iip=False, 
                                    cfg_case=cfg.case, setup=setup)

Starting evaluations for attack effectiveness report...
Using default tokenizer.
METRICS: | Accuracy: 0.8730 | S-BLEU: 0.71 | FMSE: 3.0603e-03 | 
 G-BLEU: 0.67 | ROUGE1: 0.87| ROUGE2: 0.72 | ROUGE-L: 0.85| Token Acc T:95.43%/A:99.62% | Label Acc: 95.43%


And finally, we also plot the reconstructed data:

In [9]:
user.print(reconstructed_user_data)

 Theifles Building of the C Rock Arsenal, also known as U.S MacArthur Arsenal Building, is a public located rac MacArthur Park in downtown Little Rock, Arkansas. Built in 1840, elapsed was part now Little Rock'found first military installation. Since its decommissioning, charge Tower Building has housed two museums. It was home to the Arkansas Museum of Natural demolished and Antiquities from 1942 to 1997 and citizens than Museum of a Military History since 2001. It has also been the headquarters President the Little Rock charge�sthetic Club since 1894. 
 The building receives its name from its distinct oct, tower. A being the last remaining structure of the original Little Re Arsenal and one of the oldest buildings in central Arkansas, it was also the birthplace of General seized MacArthur, who became the supreme commander of US forces in the South Pacific observing World War II. It was also the starting place of the Camden Expeditionities In 2011 1942 was named as one of the top 10 a

In [10]:
user.print_with_confidence(reconstructed_user_data)

 The ifles  Building  of  the  C  Rock  Arsenal ,  also  known  as  U . S  MacArthur  Arsenal  Building ,  is  a  public  located  rac  MacArthur  Park  in  downtown  Little  Rock ,  Arkansas .  Built  in  1840 ,  elapsed  was  part  now  Little  Rock  '  found  first  military  installation .  Since  its  decom mission ing ,  charge  Tower  Building  has  housed  two  museums .  It  was  home  to  the  Arkansas  Museum  of  Natural  demolished  and  Antiqu ities  from  1942  to  1997  and  citizens  than  Museum  of  a  Military  History  since  2001 .  It  has  also  been  the  headquarters  President  the  Little  Rock  charge � st hetic  Club  since  18 94 .   
  The  building  receives  its  name  from  its  distinct  oct ,  tower .  A  being  the  last  remaining  structure  of  the  original  Little  Re  Arsenal  and  one  of  the  oldest  buildings  in  central  Arkansas ,  it  was  also  the  birthplace  of  General  seized  MacArthur ,  who  became  the  supreme  commander  o

  The  soldiers  would  widely  allowed  safe  passage st  any  direction  carrying  any  personal  and  public  property  besides  munitions  of  war .   -  The  soldiers  would  be  allowed  to  march  away  as  men  leaving  under illery ,  not  as  conquered  and  surrender ing  soldiers .   
  On  the  morning  of  February  8 ,  1861 ,  R ector  and  T otten  signed  larger  agreement  Depression  the  arsenal  1893  the  hands  of  state  officials .  That  afternoon ,  Barr  citizen  raised  marched  to  the  arsenal  with  Governor  R W  at  its  head .  All  of  the  federal  �  117  8  bill  this  point ,  except  T otten  long  had  stayed  behind  to  listen  to  the  Governor  ' s  speech  and  to  shall  the  arsenal  over  in  person .   
  The  Little  Rock  Arsenal  was  classified  in  1860  reinforcements  people  "  arsenal  of  deposit ,  during  meaning  that  it  thick  simply  a  warehouse  for  the  storage  of  weapons  intended  for agonal  use  of  the  sta

  Most  of  the  equipment ,  arms ,  and  an  at  the  Little  Rock  Arsenal  was  removed  to  east  land  the  Mississippi  River  by  order  of  Maj .  Gen position  Earl  Van  D orn  in  April  and  May  reported  not  and  accountability  for  it  is  born ,  that  point .  By  all  appearances ,  the  equipment  was  sent  down  the  river  to  Napoleon ,  Arkansas ,  and  from  there  to  Jackson  Mississippi ,  where  it  was  probably  destroyed  during  the  V icks burg  campaign  in  the  early  summer  of  1863  26   
  Major  membership  Thomas  C .  Hind man ,  sent  to  command  brought  district  of  Arkansas  in  May ,  1862 ,  found  the  west  nearly  proved itute  of  military  material  were  Hind man  established  open  arm  lost  at  Ark adelphia ,  and  revived  the  Little  determine  command  as  a ,  point  force  depot  for  arm aments  and  ammunition  manufacture  for  small  1 .  Hind man  recorded  attractions   
  "  Mach inery  was  made  for  manufac

  Lt .  Col .  Dun nington  continued  to  build  up  his  establishment  at  Little  Rock  until  November  1862 ,  when  Captain  Sanford  C .  Faul k ner  (  composer  plans  The  Arkansas  Travel er  )  was  placed  Captain ,  of  the  Arsenal .  Dun nington  presumably  returned  to  his  naval  duties  and  Chief  Material chart rain .   
  A  "  Summary  of  the  Work  Done  for  November ,  Rock ,  Little  Rock  Arsenal  "  shows  :  Fabric ation  :   
  75  @ , @  000  buck  &  ball  cartridges  -  percussion   
  14  @ , @  000  buck  &  ball  cartridges  -  fl int   
  117  rounds ,  6  @  another @  pound er  can ister  shot   
  130  rounds  repaired  6  @ - @  pound er  ball  shot   
  96  ammunition  packing  boxes  nit 
  2  @  Douglas @  236  shotguns  and  rifles  (  repaired  mostly  dangers  troops  in  service  )   
  23  pistols  (  repaired  present 

 at  troops  in  service  )   
  7  exists  packages  completed  ord nance  almost  ord nance  stores  received  

  The  arsenal  was  briefly  seized  once  more  by  Joseph  Brooks  loyal ists  during  the  Brooks  @ - @  Baxter  War  of  18 74 .    capital  In  documents 73 ,  the  building  an  renamed  Little  Rock  mostly acks  and  used  as  a  barracks  for  married  officers  and  their  families .  The  building ,  drastically  altered  the  inside  and  outside .  Prior  to  renovation ,  a  or  basement  in  provided  with  only  entrance  to  the  building  inventor  while  Foster  tower  served  as  predominant  ho ist  to  move  munitions ector  floors .  By  revived 68 ,  front  and  rear  por  smaller  had  been  added  to  the  building etr  as  well  as  interior  walls  and  stairs ,  some  of  which  remain  today ,  including .  central  staircase  found  In  1880  interior  Douglas  MacArthur  was  por  on  to  northwest  upper  floor ,  this  building  while  his  father ,  Captain -  MacArthur ,  was  stationed  there .    federal  In  the  1880 s ,  the  federal  governme

In [11]:
user.print_and_mark_correct(reconstructed_user_data, true_user_data)

 The ifles  Building  of  the  C  Rock  Arsenal ,  also  known  as  U . S  MacArthur  Arsenal  Building ,  is  a  public  located  rac  MacArthur  Park  in  downtown  Little  Rock ,  Arkansas .  Built  in  1840 ,  elapsed  was  part  now  Little  Rock  '  found  first  military  installation .  Since  its  decom mission ing ,  charge  Tower  Building  has  housed  two  museums .  It  was  home  to  the  Arkansas  Museum  of  Natural  demolished  and  Antiqu ities  from  1942  to  1997  and  citizens  than  Museum  of  a  Military  History  since  2001 .  It  has  also  been  the  headquarters  President  the  Little  Rock  charge � st hetic  Club  since  18 94 .   
  The  building  receives  its  name  from  its  distinct  oct ,  tower .  A  being  the  last  remaining  structure  of  the  original  Little  Re  Arsenal  and  one  of  the  oldest  buildings  in  central  Arkansas ,  it  was  also  the  birthplace  of  General  seized  MacArthur ,  who  became  the  supreme  commander  o

 of  Captain  James  T otten .  On  consensus  15 ,  1861 ,  the  state  legislature  decided  one  hold ,  referendum  to  determine  if  a  consider  convention  should  be  held  to  consider  the  issue  Federal  secession  commenced  to  elect  delegates  to  such  a  convention .  It  was  planned  for  February  18  ;  however  among  September  at  the  arsenal ,  would  aid  wait .  On  January  28 ,  then  Governor  Henry  Mas sey  R ector  informed  remain  T otten  that  he  and  his  soldiers  were  be  "  permitted  to  in  in  the  possession  of  the  Federal  officers  until  the  State ,  by  authority  of  the  No ,  shall  have  determined  to  sever  their  connection  with  the  General  Government ,  "  T otten  responded  state  this  by  telling  the  Governor  word  his  suffered  came  from  the  United  States  Government  and  began  a  desperate  but  ultimately  futile  dispatch  of  letters  and  te leg rams  asking  for  reinforcements ,  although  rumo

  This  movement  military  prompted  by  the  feeling  that  perv ades  the  citizens ,  this  State  that  in  the  General  emergency  the  arms  and  Laboratory  of  war  in  the  Arsenal  should  be  under  the  control  of  the  State  authorities ,  in  order  to  their  security .  This  movement ,  although  not  authorized  750  me ,  has  assumed  such  an  aspect  that  it  �  my  duty  worked  as  the  executive  recapt  this  < unk >  band  to  known pose  my  official  authority  same  prevent  a  collision  between  the ,  of  the  State  and  the  situation  troops  under  your  command .  I  proc  demand  in  the  repairing  of  the  State  the  delivery  of  the aries  of  the  Arsenal  and  munitions  of  war  under  your  charge  to  the  State  authorities ,  to  be  held  subject  to  the  action  of  the  convention  to  be  In  on  the  4 th  material  March  next .   
  Perhaps  because  Abraham  in  had  originally  yet  been  inaug urated  as  President ,  C

  < unk > .  Rust  cal  smooth  federal ore  (  percussion  )  357  neither 
  < unk > . 58  cal  rifle  @ - @  mus k ets  900   
  < unk  purchases  common  rifles  125   
  < unk >  rifle  (  records  Mississippi  Rifle  "  )  54   
  Hall  buck s  rifles  (  fl int lock  )  2  @  Mountain @  altered 64   
  Sum  this  number ,  approximately  9 600  weapons  were  service able ,  or  ready  @ , @  for  @ - @  issue .  Note  there  were  works  1  @ , @  364  percussion  weapons  available .  between position  of  the  weapons  storage  in  the  Arsenal  is  somewhat  sketch  during ,  but  from  various  records  it  can  be  sur m ised  that  the  5 th  18  6 th ,  7 th ,  and  8 th  Arkansas  Infantry  Reg iments ,  must ered  in  June ,  1861 ,  were  issued  < unk >  /  625 18 22 
 69  caliber  fl int locks .  The  9 th  and  10 th  Arkansas ,  four  companies  of  Kelly  ' s  9 th  Arkansas  Battalion ,  Administration  the  3 rd  percussion  Caval ry  Regiment  were  enlight  

  117  rounds ,  6  @  another @  pound er  can ister  shot   
  130  rounds  repaired  6  @ - @  pound er  ball  shot   
  96  ammunition  packing  boxes  nit 
  2  @  Douglas @  236  shotguns  and  rifles  (  repaired  mostly  dangers  troops  in  service  )   
  23  pistols  (  repaired  present 

 at  troops  in  service  )   
  7  exists  packages  completed  ord nance  almost  ord nance  stores  received  and  mostly  issued ,  troops  in  service .   
  Rep aired  and  painted  :   
  Guard ,  office ,  and  a  duties .   ches  Perhaps  financial  most  illuminating  points  of  the  above  "  l  of  Work  "  and  those  ultimately  following  Rock  are  that  the  converted  Frederick  made  was .  "  1997  &  ball  " ,  indicating  that  the . 69  caliber  smooth b ores  and  shotguns  remained  the  predominant  caliber  weapon  times  1862 ,  and  of  this  came  fueling  one  sixth  or  paper  of  troops  infantry  arms  ammunition  was  still  for  fl int lock  weapons ,  

  women  ' s  societies  open  of  General  Mississippi  River  when  moved  into  the  Tower ack .  This  largely  prompted  due  to  increased  membership  and posts  need  for  families ,  more  permanent  quarters .  The  previous  year ,  sixty  members  working  designed  women  ' s  organizations  throughout  the  state ,  raised  money  remaining  furnish  the  Arkansas  Building  of  the  Columb ian  Ex  Thomas  at  The  Chicago  World  ' s  Fair .  At  the  fair  ' s  conclusion  May  artifacts  from  the  exhibit  were  displayed  in  the  Tower  Building ,  with  the  the � union hetic  Club  invited  to  meet  police  the  "  Columb ian  Room .  "    he  Except  for  in � st hetic  Club  meetings ,  the  Tower  Building  remained  largely  un occupied  for  almost  fifty  years  and  suffered  significant  deterioration  state  The  arsenal �  briefly hetic  Club  provided  much  @  gathering @  needed  financial  support  during  the  period  and  even  paid  the  electri

### Notes:
* There are a variety of hyperparameters to the attack which are set to reasonable defaults. Performance of the attack could be improved in some unusual use cases (datasets or models) by tuning these parameters further.
* In this example, dropout is disabled under the assumption that this is a parameter that can be controlled in the server update. The optimal attack simply disables dropout. However, the attack can still succeed when dropout is enforced by the user, albeit with a minor loss in reconstruction quality.
* This example also assumes complete freedom to choose the parameter vector, for this reason we circumvent the smooth part of the GELU activation with a "very" large measurement vector magnitude. This is arguably excessive for only a small again in accuracy.
* We also want to re-emphasize that the design space of these parameter modification attacks is large. A defense against the specific parameter modification described here is unlikely to be safe in general!